In [1]:
############################################################################################################
# Overview: This script cleans the extracted patient-level data mart.
############################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import datetime
import gc
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import warnings
from ast import literal_eval
from itertools import zip_longest
from pyarrow.parquet import ParquetFile
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [1]:
########################################################################################################################
# USER_SPECIFIC SETTING
# DATA_IN_DIR_PATH: Path of the input directory of the patient-level dataset 
# (created in C01_Data_Transfer.ipynb)
# DATA_OUT_DIR_PATH: Path of the output directory of the patient-level dataset 
# (created in C01_Data_Transfer.ipynb)
# DICT_IN_FILE_PATH: Path of the input data dictionary file
# (created in C03_Encode_Encounter_Variables.ipynb)
# DICT_OUT_FILE_PATH: Path of the output data dictionary file
########################################################################################################################
DATA_IN_DIR_PATH: str = '../00_Data/00_Raw_Data/'
DATA_OUT_DIR_PATH: str = '../00_Data/01_Cleaned_Data/'
DICT_IN_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v2.xlsx'
DICT_OUT_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v3.xlsx'

In [ ]:
########################################################################################################################
# Load the data dictionary
########################################################################################################################
df_dict_pat: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Patient')
df_dict_enc: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Encounter')

In [ ]:
########################################################################################################################
# Load the header of the patient-level dataset
########################################################################################################################
pat_data_path: str = os.path.join(DATA_IN_DIR_PATH, 'Patient_full.parquet')
pf: ParquetFile = ParquetFile(pat_data_path)
first_row: pa.lib.RecordBatch = next(pf.iter_batches(batch_size=1))
header: list[str] = pa.Table.from_batches([first_row]).to_pandas().columns.to_list()

In [ ]:
########################################################################################################################
# Initiate a single-column pandas.DataFrame containing the first column of the patient-level dataset
# (i.e., the patient identifier column = PatientDurableKey)
# Create an extra copy of df_dict_pat for modification
########################################################################################################################
df_out: pd.DataFrame = pd.DataFrame({header[0]: pd.read_parquet(pat_data_path, 
                                                                columns=[header[0]]).iloc[:, 0]})
N: int = df_out.shape[0]    
df_dict_pat_v2: pd.DataFrame = pd.DataFrame(None, columns=df_dict_pat.columns)
df_dict_pat_v2.insert(2, 'Sampled_Percentage', np.nan)

In [ ]:
########################################################################################################################
# Part A. Encode and extract data of date & continuous variables
########################################################################################################################
for row_idx, row in df_dict_pat[df_dict_pat['Variable_Type'].isin(['datetime.date',
                                                                   'Date as float',
                                                                   'Continuous'])].iterrows():
    var_name: str = row['Variable_Name']
    var_type: str = row['Variable_Type']
    remark: str = row['Remark']
    log_head: str = f'[{row_idx}] {var_name} '

    ####################################################################################################################
    # A1. Load the uncleared single-column data
    ####################################################################################################################
    df_cur: pd.DataFrame = pd.read_parquet(pat_data_path, columns=[var_name])

    ####################################################################################################################
    # A2. Extract data
    ####################################################################################################################
    # Case 1. Date variables in datetime.date format
    if var_type == 'datetime.date':
        print(f'{log_head}datetime.date --> Date')
        df_cur[var_name] = pd.to_datetime(df_cur[var_name])
        new_var_type: str = 'Date'

    # Case 2. Date variables in float format
    elif var_type == 'Date as float':
        print(f'{log_head}Date as float --> Date')
        df_cur[var_name] = df_cur[var_name].apply(lambda x: str(int(x)) if pd.notna(x) else None)
        df_cur[var_name] = pd.to_datetime(df_cur[var_name], format='%Y%m%d', errors='coerce')
        new_var_type: str = 'Date'

    # Case 3. Continuous variables
    else:
        print(f'{log_head}Continuous (as is)')
        new_var_type: str = 'Continuous'

    df_out[var_name] = df_cur[var_name]

    ########################################################################################################################
    # A3. Update dictionary
    ########################################################################################################################
    sampled_ratio: int = int(round(df_cur[var_name].notna().sum() / N, 2) * 100)
    record_cur: pd.DataFrame = pd.DataFrame({'Variable_Name': [var_name],
                                             'Sample_Size': int(df_cur[var_name].notna().sum()),
                                             'Sampled_Percentage': [sampled_ratio],
                                             'Encoded_Values': [np.nan],
                                             'Variable_Type': [new_var_type],
                                             'Remark': [remark]})
    df_dict_pat_v2 = pd.concat([df_dict_pat_v2, record_cur], ignore_index=True)
    del df_cur
    gc.collect() 


In [ ]:
########################################################################################################################
# Part B. Encode race data (from 5 variables) into a single nominal variable
########################################################################################################################
# Initialize a list to store race data
race_records: list[list[int]] = []

########################################################################################################################
# B1. Load the uncleaned 5-column data
########################################################################################################################
race_vars: list[str] = ['FirstRace', 'SecondRace', 'ThirdRace', 'FourthRace', 'FifthRace']
df_cur: pd.DataFrame = pd.read_parquet(pat_data_path, columns=race_vars)

########################################################################################################################
# B2. Load and apply the encoding scheme to each race column, then concatenate
########################################################################################################################
race_encoder: dict[str, int] = literal_eval(
    df_dict_pat.loc[df_dict_pat['Variable_Name'] == 'FirstRace',
                    'Encoded_Values'].values[0]
)

for var in race_vars:
    df_cur[var] = df_cur[var].replace(race_encoder)
    race_records.append(df_cur[var].values)

########################################################################################################################
# B3. Transform race_records
########################################################################################################################
race_records = [row.tolist() for row in np.array(race_records).T]
race_records_new: list[int] = []

for race_record in race_records:
    unique_races: np.array = np.unique(np.array(race_record)[~np.isnan(race_record)])
    if len(unique_races) == 1:              # Take the unique race
        race_records_new.append(unique_races[0])
    elif len(unique_races) > 1:              # Report multiple races
        race_records_new.append(6)
    else:                                    # Didn't report
        race_records_new.append(np.nan)

########################################################################################################################
# B4. Obtain the decoder from the encoder
########################################################################################################################
race_decoder: dict[int, str] = {v: k for k, v in race_encoder.items() if k != ''} | {6: 'Multiple Races'}

########################################################################################################################
# B5. Use race_records_new to perform one-hot encoding and concatenate to df_out, and update the data dictionary
########################################################################################################################
df_race: pd.DataFrame = pd.DataFrame({'Race': race_records_new})
assert set(df_race['Race'].dropna().unique()) == set(range(7))

for race_code, race_desc in race_decoder.items():
    new_var_name: str = f'Race{race_code}_{race_desc}'
    df_out[new_var_name] = df_race['Race'].apply(lambda x: np.nan if pd.isna(x) else int(x == race_code)).astype('Int32')
    sample_size: int = df_out[new_var_name].notna().sum()
    sampled_ratio: int = int(round(sample_size / N, 2) * 100)

    record_cur: pd.DataFrame = pd.DataFrame({
        'Variable_Name': [new_var_name],
        'Sample_Size': [sample_size],
        'Sampled_Percentage': [sampled_ratio],
        'Encoded_Values': [{0: 'No', 1: 'Yes'}],
        'Variable_Type': ['Binary'],
        'Remark': [np.nan]
    })

    df_dict_pat_v2 = pd.concat([df_dict_pat_v2, record_cur], ignore_index=True)

del race_records
del race_records_new
del df_race
gc.collect()

In [ ]:
########################################################################################################################
# Part C. Encode all other variables (binary / nominal / ordinal)
########################################################################################################################
df_dict_pat_sub: pd.DataFrame = df_dict_pat[df_dict_pat['Variable_Type'].isin(['Nominal', 'Binary', 'Ordinal'])]
df_dict_pat_sub = df_dict_pat_sub[df_dict_pat_sub['Variable_Name'].apply(lambda x: not x.endswith('Race'))]
df_out_cat: pd.DataFrame = pd.DataFrame(None)       # Create empty pandas.DataFrame for the data and dictionary of
df_dict_pat_cat: pd.DataFrame = pd.DataFrame(None)  # the categorical variables

for row_idx, row in df_dict_pat_sub.iterrows():
    var_name: str = row['Variable_Name']
    var_type: str = row['Variable_Type']
    encoder: dict[str, int] = literal_eval(row['Encoded_Values'])
    remark: str = row['Remark']
    log_head: str = f'[{row_idx}. {var_name}]'

    ####################################################################################################################
    # C1. Load the uncleaned single-column data
    ####################################################################################################################
    df_cur: pd.DataFrame = pd.read_parquet(pat_data_path, columns=[var_name])

    ####################################################################################################################
    # C2. Identify the codes for missingness
    ####################################################################################################################
    missing_encoder: dict[str, int] = {k: v for k, v in encoder.items() if v is not None and v < 0}
    non_missing_encoder: dict[str, int] = {k: v for k, v in encoder.items() if v is not None and v >= 0}
    missing_decoder: dict[int, str] = {v: k for k, v in missing_encoder.items()}
    non_missing_decoder: dict[int, str] = {v: k for k, v in non_missing_encoder.items()}

    ####################################################################################################################
    # C3. Apply the encoder to the data
    ####################################################################################################################
    df_cur[var_name].replace(encoder, inplace=True)

    ########################################################################################################################
    # C4. Handle non-ordinal variables first
    ########################################################################################################################
    if var_type in ['Binary', 'Nominal']:
    
        if var_type == 'Binary':
            assert len(non_missing_encoder) == 2
            assert set(non_missing_encoder.values()) == {0, 1}, non_missing_encoder.values()
        else:
            assert len(non_missing_encoder) > 2
    
        ####################################################################################################################
        # 4.1 Encode only the positive value for binary variables, and all values for nominal variables
        ####################################################################################################################
        for k, v in non_missing_decoder.items():
    
            if var_type == 'Binary' and k == 0:
                continue
    
            new_var_name: str = f'{var_name}_{k}={v}'
            df_out_cat[new_var_name] = df_cur[var_name].apply(
                lambda x: np.nan if pd.isna(x) or x < 0 else int(x == k)
            ).astype('Int32')  # Cleaned data
    
            sample_size: int = df_out_cat[new_var_name].notna().sum()
            sampled_ratio: int = int(round(sample_size / N, 2) * 100)
    
            record_cur: pd.DataFrame = pd.DataFrame({
                'Variable_Name': [new_var_name],
                'Sample_Size': [sample_size],
                'Sampled_Percentage': [sampled_ratio],
                'Encoded_Values': [{0: 'No', 1: 'Yes'}],
                'Variable_Type': ['Binary'],
                'Remark': [remark]
            })
    
            df_dict_pat_cat = pd.concat([df_dict_pat_cat, record_cur], ignore_index=True)  # Cleaned dictionary
    
    ########################################################################################################################
    # C5. Handle nominal encoding
    ########################################################################################################################
    else:
        assert len(non_missing_encoder) > 2
    
        ####################################################################################################################
        # 5.1 Encode only the non-negative values
        ####################################################################################################################
        df_out_cat[var_name] = df_cur[var_name].apply(
            lambda x: np.nan if pd.isna(x) or x < 0 else x
        ).astype('Int32')
    
        sample_size: int = df_out_cat[var_name].notna().sum()
        sampled_ratio: int = int(round(sample_size / N, 2) * 100)
    
        record_cur: pd.DataFrame = pd.DataFrame({
            'Variable_Name': [var_name],
            'Sample_Size': [sample_size],
            'Sampled_Percentage': [sampled_ratio],
            'Encoded_Values': [non_missing_decoder],
            'Variable_Type': ['Ordinal'],
            'Remark': [remark]
        })
    
        df_dict_pat_cat = pd.concat([df_dict_pat_cat, record_cur], ignore_index=True)  # Cleaned dictionary

    ########################################################################################################################
    # C6. Handling the encoding of the missing values
    # All missing values in these newly created columns will be filled as 0 to represent structural missingness
    ########################################################################################################################
    for k, v in missing_decoder.items():
        new_var_name: str = f'{var_name}_{k}={v}'
        df_out_cat[new_var_name] = df_cur[var_name].apply(
            lambda x: np.nan if pd.isna(x) else int(x == k)
        ).astype('Int32')
    
        sample_size: int = df_out_cat[new_var_name].notna().sum()
        sampled_ratio: int = int(round(sample_size / N, 2) * 100)
    
        record_cur: pd.DataFrame = pd.DataFrame({
            'Variable_Name': [new_var_name],
            'Sample_Size': [sample_size],
            'Sampled_Percentage': [sampled_ratio],
            'Encoded_Values': [{0: 'No', 1: 'Yes'}],
            'Variable_Type': ['Binary'],
            'Remark': [remark]
        })
    
        df_dict_pat_cat = pd.concat([df_dict_pat_cat, record_cur], ignore_index=True)  # Cleaned dictionary
    
    del df_cur
    gc.collect()

In [ ]:
########################################################################################################################
# Part D. Concatenate the data sets and dictionaries respectively, and save them
########################################################################################################################
df_out = pd.concat([df_out, df_out_cat], axis=1)
df_dict_pat_v2 = pd.concat([df_dict_pat_v2, df_dict_pat_cat], axis=0)

os.makedirs(DATA_OUT_DIR_PATH, exist_ok=True)
data_file_path: str = os.path.join(DATA_OUT_DIR_PATH, 'Patient_full_v1.parquet')
df_out.to_parquet(data_file_path)
print(f'Cleaned data (v1) saved with dimension={df_out.shape}')

with pd.ExcelWriter(DICT_OUT_FILE_PATH) as writer:
    df_dict_pat_v2.to_excel(writer, sheet_name='Patient', index=False)
    df_dict_enc.to_excel(writer, sheet_name='Encounter', index=False)
print(f'Patient-level dictionary updated with {df_dict_pat_v2.shape[0]} variables.')

# The data dictionary should contain 1 variable (PatientDurableKey) less than the data.